In [1]:
!pip install -q transformers accelerate peft bitsandbytes datasets trl 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 87.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"

# Cấu hình 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.pad_token_id = tokenizer.pad_token_id

2025-11-18 06:59:41.447017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763449181.672666      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763449181.729102      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

NameError: name 'model' is not defined

In [4]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 14,966,784 || all params: 3,100,905,472 || trainable%: 0.4827


In [ ]:
import os
import glob
import json
from datasets import Dataset
# Đọc file JSONL
data = []
with open("/kaggle/input/train-no-reasoning-full/train.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(eval(line))

# Chuyển thành Dataset
train_dataset = Dataset.from_list(data)

def format_prompt(ex):
    messages = ex["messages"]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

train_dataset = train_dataset.map(format_prompt)


Map:   0%|          | 0/2641 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset[0]["text"])

In [7]:
def tokenize_function(examples):
    output = tokenizer(
        examples["text"],
        truncation=True,
        padding=False,
        max_length=2048,
        return_tensors=None,
    )
    output["labels"] = output["input_ids"].copy()
    return output

train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names,
)

Map:   0%|          | 0/2641 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer, SFTConfig
sft_config = SFTConfig(
    output_dir="./qwen--no-reasoning-lora",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,

    packing=True,         
    gradient_checkpointing=True,

    bf16=False,
    fp16=True,

    dataset_text_field="text",
    report_to="none",
)


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [9]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    args=sft_config,
    peft_config=peft_config,
)

trainer.train()




/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Packing train dataset:   0%|          | 0/2641 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,0.963100
20,0.472300
30,0.290100
40,0.294800
50,0.284000
60,0.260100
70,0.254500
80,0.261800
90,0.274000
100,0.274900


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=330, training_loss=0.26718148426576094, metrics={'train_runtime': 11018.337, 'train_samples_per_second': 0.477, 'train_steps_per_second': 0.03, 'total_flos': 7.818017999939174e+16, 'train_loss': 0.26718148426576094, 'epoch': 2.0})

In [10]:
trainer.save_model()

In [11]:
!zip -r /kaggle/working/qwen-reasoning-lora.zip /kaggle/working/qwen-reasoning-lora


  adding: kaggle/working/qwen-reasoning-lora/ (stored 0%)
  adding: kaggle/working/qwen-reasoning-lora/added_tokens.json (deflated 67%)
  adding: kaggle/working/qwen-reasoning-lora/README.md (deflated 45%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/ (stored 0%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/added_tokens.json (deflated 67%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/rng_state.pth (deflated 25%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/scheduler.pt (deflated 56%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/README.md (deflated 65%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/tokenizer.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 81%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/adapter_model.safetensors (deflated 8%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/training_args.bin (deflated 51%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/chat_template.jinja (deflated 71%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/tokenizer_config.json (deflated 89%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/optimizer.pt (deflated 8%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/adapter_config.json (deflated 56%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/scaler.pt (deflated 60%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/merges.txt (deflated 57%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/vocab.json (deflated 61%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/special_tokens_map.json (deflated 69%)
  adding: kaggle/working/qwen-reasoning-lora/checkpoint-330/t

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load base model 4-bit
base_model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"
adapter_path = "/kaggle/working/qwen-no-reasoning-lora"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Load adapter
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048)
        (layers): ModuleList(
          (0-35): 36 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.L

In [24]:
problem_text = r'''
You are given a string target, an array of strings words, and an integer array costs, both arrays of the same length.
Imagine an empty string s.
You can perform the following operation any number of times (including zero):

Choose an index i in the range [0, words.length - 1].
Append words[i] to s.
The cost of operation is costs[i].

Return the minimum cost to make s equal to target. If it's not possible, return -1.
 
Example 1:

Input: target = "abcdef", words = ["abdef","abc","d","def","ef"], costs = [100,1,1,10,5]
Output: 7
Explanation:
The minimum cost can be achieved by performing the following operations:

Select index 1 and append "abc" to s at a cost of 1, resulting in s = "abc".
Select index 2 and append "d" to s at a cost of 1, resulting in s = "abcd".
Select index 4 and append "ef" to s at a cost of 5, resulting in s = "abcdef".


Example 2:

Input: target = "aaaa", words = ["z","zz","zzz"], costs = [1,10,100]
Output: -1
Explanation:
It is impossible to make s equal to target, so we return -1.

 
Constraints:

1 <= target.length <= 2000
1 <= words.length == costs.length <= 50
1 <= words[i].length <= target.length
target and words[i] consist only of lowercase English letters.
1 <= costs[i] <= 105


Boilerplate code:
```python
def minimumCost(target, words, costs):
    ...
```
'''

In [ ]:
system_prompt = (
"You are an expert Python programmer and problem solver. "
"You will be given LeetCode-style problems and must generate structured Python code solutions. "
"Always return a class named 'Solution' with the required method, and include an alias function "
"outside the class that calls the method. Ensure your code includes detailed comments and "
"follows Python best practices."
)

user_prompt = problem_text + (
"\n\nPlease implement the solution using a class named Solution "
"with the method matching the function signature. "
"Include an alias function outside the class that calls the method."
)



In [ ]:
def format_messages(system_prompt, user_prompt):
    return (
        "<|im_start|>system\n" + system_prompt + "<|im_end|>\n"
        "<|im_start|>user\n" + user_prompt + "<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

# Format prompt
prompt = format_messages(system_prompt, user_prompt)

inputs = tokenizer(
    prompt,
    return_tensors="pt"
).to(model.device)

with torch.inference_mode():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
    )

# Decode
generated = tokenizer.decode(
    output_ids[0][inputs["input_ids"].shape[1]:],  # only new tokens
    skip_special_tokens=True
)

print(generated)